In [1]:
from utils.structure import *
from utils.predicter import *
from utils.solver import *

In [2]:
# build structure and display
test_datas = open('./data/test_demo.txt', 'r')
test_datas = test_datas.readlines()

In [3]:
# input test
for test_data in test_datas[:5]:
    s = Structure(test_data)
    print(s.data)
    print(s.display())

['1', '.', '.', '4', '3', '4', '1', '.', '.', '3', '2', '1', '2', '.', '.', '3']
+---+---+
|1 .|. 4|
|   |   |
|3 4|1 .|
+---+---+
|. 3|2 1|
|   |   |
|2 .|. 3|
+---+---+

['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '6', '.', '9', '3', '.', '.', '.', '9', '.', '.', '7', '6', '.', '.', '.', '4', '4', '.', '.', '.', '.', '6', '.', '3', '.', '.', '.', '.', '8', '.', '.', '.', '.', '2', '.', '1', '.', '.', '.', '.', '8', '5', '.', '7', '.', '.', '6', '5', '.', '.', '4', '.', '.', '8', '4', '.', '.', '.', '9', '.', '.', '.', '.', '3', '2', '.', '.', '.', '.', '.']
+-----+-----+-----+
|. . .|. . .|. . .|
|     |     |     |
|. . 6|. 9 3|. . .|
|     |     |     |
|9 . .|7 6 .|. . 4|
+-----+-----+-----+
|4 . .|. . 6|. 3 .|
|     |     |     |
|. . .|8 . .|. . 2|
|     |     |     |
|. 1 .|. . .|8 5 .|
+-----+-----+-----+
|7 . .|6 5 .|. 4 .|
|     |     |     |
|. 8 4|. . .|9 . .|
|     |     |     |
|. . 3|2 . .|. . .|
+-----+-----+-----+

['.', '5', '2', '.', '.', '9', '.', '.', 

In [4]:
# scan all area test
for i in range(6, 10):
    print(i, ':')
    s = Structure(test_datas[i])
    # print(s.display())
    solver = BasicSolver(s)
    print('- Before ', solver.scan_all(method='area'))
    print('- Processing ', solver.scan_all())
    print('- After ', solver.scan_all(method='area'))

6 :
- Before  False
- Processing  False
- After  True
7 :
- Before  True
- Processing  False
- After  True
8 :
- Before  True
- Processing  False
- After  True
9 :
- Before  False
- Processing  False
- After  True


In [5]:
# scan all group test
for i in range(11, 16):
    print(i, ':')
    s = Structure(test_datas[i])
    # print(s.display())
    solver = BasicSolver(s)
    print('- Before ', solver.scan_all(method='group'))
    print('- Processing ', solver.scan_all())
    print('- After ', solver.scan_all(method='group'))

11 :
Before grouped drop, scanned drop can also make some changes.
- Before  True
- Processing  False
- After  False
12 :
Before grouped drop, scanned drop can also make some changes.
Before grouped drop, scanned drop can also make some changes.
Before grouped drop, scanned drop can also make some changes.
- Before  True
- Processing  False
- After  False
13 :
Before grouped drop, scanned drop can also make some changes.
- Before  True
- Processing  False
- After  False
14 :
Before grouped drop, scanned drop can also make some changes.
Before grouped drop, scanned drop can also make some changes.
- Before  True
- Processing  False
- After  False
15 :
Before grouped drop, scanned drop can also make some changes.
Before grouped drop, scanned drop can also make some changes.
Before grouped drop, scanned drop can also make some changes.
Before grouped drop, scanned drop can also make some changes.
- Before  False
- Processing  False
- After  False


In [6]:
# scan all square test
for i in range(17, 22):
    print(i, ':')
    s = Structure(test_datas[i])
    # print(s.display())
    solver = BasicSolver(s)
    print('- Before ', solver.scan_all(method='square'))
    print('- Processing ', solver.scan_all())
    print('- After ', solver.scan_all(method='square'))

17 :
- Before  False
- Processing  False
Found the group drop part is ({0, 1}, {0, 1})  rows.
Found the group drop part is ({8, 5}, {8, 5})  cols.
Found the group drop part is ({8, 2}, {8, 2})  cols.
- After  True
18 :
- Before  False
- Processing  False
Found the group drop part is ({8, 4}, {8, 4})  rows.
Found the group drop part is ({0, 2}, {0, 2})  rows.
Found the group drop part is ({0, 4}, {0, 4})  cols.
Found the group drop part is ({4, 7}, {4, 7})  rows.
Found the group drop part is ({0, 1}, {0, 1})  cols.
- After  True
19 :
- Before  False
- Processing  False
Found the group drop part is ({3, 4}, {3, 4})  rows.
- After  True
20 :
- Before  False
- Processing  False
Found the group drop part is ({0, 2}, {0, 2})  rows.
Found the group drop part is ({2, 3}, {2, 3})  cols.
Found the group drop part is ({2, 4}, {2, 4})  rows.
Found the group drop part is ({3, 4}, {3, 4})  cols.
- After  True
21 :
- Before  False
- Processing  False
Found the group drop part is ({8, 7}, {8, 7})  col

In [7]:
solver.steps

[]

In [ ]:
tmp = "	 	 	6	 	 	4	1	7	2\
	 	 	 	2	 	1	5	 	 \
	 	1	 	 	3	 	9	 	 \
	 	2	8	 	 	 	3	 	4\
	1	 	 	 	 	 	2	 	7\
	6	 	 	 	 	 	8	1	 \
	 	 	3	 	7	 	4	 	 \
	 	 	1	3	 	5	7	 	 \
	7	9	 	4	 	 	6	 	 "

In [ ]:
tmp = tmp.replace(' ', '.').replace('×', '.').replace('?','.').replace('	', '')
print(len(','.join(list(tmp))))
','.join(list(tmp))